In [1]:
# Imports:
import numpy as np
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import sklearn
from sklearn.metrics import mean_squared_error
import plotly.express as px
from scipy import stats


In [2]:
#set seed for reproducibility
random_state = 42
np.random.seed(random_state)

In [3]:
# same dataset as bidirectional, which is why the path is the same
with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_1_period_dataset_dict.pickle', 'rb') as f:
    us_data_dict = pickle.load(f)

In [4]:
us_data = pd.read_csv(r'/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/resources/hrnn_cpi_dataset.csv')

--------

### Horizon Analyses:

In [5]:
# import horizon files 
with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/reversed_hgru_with_corr_1_period/data/hor1_test_dataset.pickle', 'rb') as f:
    us_hor_1_test_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/reversed_hgru_with_corr_1_period/data/hor2_test_dataset.pickle', 'rb') as f:
    us_hor_2_test_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/reversed_hgru_with_corr_1_period/data/hor3_test_dataset.pickle', 'rb') as f:
    us_hor_3_test_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/reversed_hgru_with_corr_1_period/data/hor4_test_dataset.pickle', 'rb') as f:
    us_hor_4_test_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/reversed_hgru_with_corr_1_period/data/hor8_test_dataset.pickle', 'rb') as f:
    us_hor_8_test_dict = pickle.load(f)

In [6]:
us_hor_1_test_dict['All items'].shape

(33, 14)

In [7]:
import sys        
sys.path.append('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/hgru_model/model')       
from GRU_model import *
from config import * 
from utils import *
Lr = 0.04329823452489634

sys.path.append('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/hgru_model')       
from pipeline_config import *

2023-11-14 16:26:11.327695: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
num_forecast_steps = 8

category_horizon_dict = {}

for category_name in list(us_hor_1_test_dict.keys()):
    # load model
    cat_model = GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb)
    cat_optimizer = torch.optim.AdamW(cat_model.parameters(), lr=Lr)
    cat_model.to(Device)

    category_model, optimizer, checkpoint, valid_loss_min = load_checkpoint('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/hgru_model/models_weights/'+category_name+'.pt', cat_model, cat_optimizer)

    horizon_dict = {}

    # Generate forecasts step by step
    for step in range(num_forecast_steps):
        with torch.no_grad():
            # create dataloader
            x_test = us_hor_1_test_dict[category_name].iloc[:,:-1].to_numpy()
            y_test= us_hor_1_test_dict[category_name].iloc[:,-1:].to_numpy()

            x_test = torch.from_numpy(x_test).to(torch.float32)
            y_test = torch.from_numpy(y_test).to(torch.float32)

            test_dataset = TensorDataset(x_test, y_test)

            test_dataloader =  DataLoader(test_dataset, batch_size=33, shuffle=False) #### NOTE: I had to fix the batch size to 33 because of size mismatch issues (input was larger than batch size and it caused issues - so it is 33 rather than 32 like the rest (canada and norway datasets))

            for inputs, labels in test_dataloader:
                # since each batch contains 32 samples, and the largest df is of size 32, then we dont need to concat the outputs per batch per category.
                inputs = inputs.view(inputs.shape[0], SequenceLength, Features)
                #print(f'inputs shape: {inputs.shape}')
                #print(f'labels shape: {labels.shape}')
                inputs, labels = inputs.to(Device), labels.to(Device)
                out = category_model(inputs)
                out_df = pd.DataFrame(out)
                
                first = us_hor_1_test_dict[category_name].iloc[:,1:].reset_index(drop = True).copy()
                print(f'first shape: {first.shape}')
                second = out_df.reset_index(drop = True).rename(columns = {0: 'Inflation t+'+str(2+step)})
                print(f'second shape: {second.shape}')

                us_hor_1_test_dict[category_name] = pd.concat([first, second],axis=1) #prediction

                horizon_dict[step+2] = us_hor_1_test_dict[category_name]

    category_horizon_dict[category_name] = horizon_dict
    


first shape: (33, 13)
second shape: (33, 1)
first shape: (33, 13)
second shape: (33, 1)
first shape: (33, 13)
second shape: (33, 1)
first shape: (33, 13)
second shape: (33, 1)
first shape: (33, 13)
second shape: (33, 1)
first shape: (33, 13)
second shape: (33, 1)
first shape: (33, 13)
second shape: (33, 1)
first shape: (33, 13)
second shape: (33, 1)
first shape: (33, 13)
second shape: (33, 1)
first shape: (33, 13)
second shape: (33, 1)
first shape: (33, 13)
second shape: (33, 1)
first shape: (33, 13)
second shape: (33, 1)
first shape: (33, 13)
second shape: (33, 1)
first shape: (33, 13)
second shape: (33, 1)
first shape: (33, 13)
second shape: (33, 1)
first shape: (33, 13)
second shape: (33, 1)
first shape: (33, 13)
second shape: (33, 1)
first shape: (33, 13)
second shape: (33, 1)
first shape: (33, 13)
second shape: (33, 1)
first shape: (33, 13)
second shape: (33, 1)
first shape: (33, 13)
second shape: (33, 1)
first shape: (33, 13)
second shape: (33, 1)
first shape: (33, 13)
second sha

In [9]:
def avg_rmse_hor(prediction_dict, actual_dict, horizon):
    mse_lst = []
    for key in list(actual_dict.keys()):
        predictions = prediction_dict[key][horizon].iloc[:,-1:].reset_index(drop = True)
        actuals = actual_dict[key].iloc[:,-1:].reset_index(drop = True)

        y_pred = predictions.values
        y_actual = actuals.values

        non_null_indices = ~np.isnan(y_pred) & ~np.isnan(y_actual)

        predicted_values_non_null = y_pred[non_null_indices]
        observed_values_non_null = y_actual[non_null_indices]

        #print(f'y pred: {predicted_values_non_null}')
        #print(f'y actual: {observed_values_non_null}')
        mse = mean_squared_error(predicted_values_non_null, observed_values_non_null)
        mse_lst.append(mse)
    
    rmse_list = list(map(np.sqrt,mse_lst))
    avg_rmse = np.mean(rmse_list)
    rmse_std = np.std(rmse_list)
    
    print(f'RMSE:  {avg_rmse}')
    print(f'MSE std:  {rmse_std}')
    print(f'interval: {[avg_rmse-rmse_std, avg_rmse+rmse_std]}')

    return avg_rmse,rmse_std

In [10]:
avg_rmse_hor(category_horizon_dict, us_hor_2_test_dict, 2)

RMSE:  1.5265929542955703
MSE std:  1.2958338857007972
interval: [0.23075906859477313, 2.8224268399963677]


(1.5265929542955703, 1.2958338857007972)

In [11]:
avg_rmse_hor(category_horizon_dict, us_hor_3_test_dict, 3)

RMSE:  1.509621308629277
MSE std:  1.2142373837566842
interval: [0.2953839248725927, 2.723858692385961]


(1.509621308629277, 1.2142373837566842)

In [12]:
avg_rmse_hor(category_horizon_dict, us_hor_4_test_dict, 4)

RMSE:  1.5520421758218188
MSE std:  1.3816103492728236
interval: [0.17043182654899525, 2.9336525250946424]


(1.5520421758218188, 1.3816103492728236)

In [13]:
avg_rmse_hor(category_horizon_dict, us_hor_8_test_dict, 8)

RMSE:  1.5471566491769675
MSE std:  1.4166298022819086
interval: [0.1305268468950589, 2.9637864514588763]


(1.5471566491769675, 1.4166298022819086)

In [14]:
#category_horizon_dict - prediction dict per category per horizon
# us_hor_x_test_dict - actual dict per category for horizon x

In [15]:
with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_2_period_dataset_dict.pickle', 'rb') as f:
    hor_2_raw_dataset_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_3_period_dataset_dict.pickle', 'rb') as f:
    hor_3_raw_dataset_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_4_period_dataset_dict.pickle', 'rb') as f:
    hor_4_raw_dataset_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_8_period_dataset_dict.pickle', 'rb') as f:
    hor_8_raw_dataset_dict = pickle.load(f)

In [16]:
def create_test_dataframe(raw_dataset_dict: dict, horizon:int):
    test_dict = {}
    for key in raw_dataset_dict.keys():
        df = raw_dataset_dict[key][['Category', 'Date', 'Year', 'Inflation t+'+str(horizon)]]
        df.dropna(inplace=True)
        df.rename(columns={'Inflation t+'+str(horizon): 'Actual Horizon '+str(horizon)}, inplace=True)
        target_df = df[df['Year'] > Year]
        target_df = target_df.reset_index(drop = True)
        test_dict[key] = target_df
    return test_dict

In [17]:
test_dict_hor_2 = create_test_dataframe(hor_2_raw_dataset_dict, 2)
test_dict_hor_3 = create_test_dataframe(hor_3_raw_dataset_dict, 3)
test_dict_hor_4 = create_test_dataframe(hor_4_raw_dataset_dict, 4)
test_dict_hor_8 = create_test_dataframe(hor_8_raw_dataset_dict, 8)


/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_21884/3575878633.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_21884/3575878633.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Inflation t+'+str(horizon): 'Actual Horizon '+str(horizon)}, inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_21884/3575878633.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [18]:
def get_df_with_predictions_horizon(prediction_dic: dict, actual_dic:dict, horizon: int) -> dict:
    all_data_dict = {}
    for key in list(prediction_dic.keys()):
        predictions = prediction_dic[key][horizon].iloc[:,-1:].reset_index(drop = True)
        predictions = predictions.rename(columns = {f'Inflation t+{horizon}': f'Prediction Horizon {horizon}'})
        actuals = actual_dic[key]

        data = pd.concat([predictions, actuals], axis=1).loc[:,['Category','Date','Year',f'Prediction Horizon {horizon}', f'Actual Horizon {horizon}']]
        all_data_dict[key] = data
        
    return all_data_dict

In [19]:
all_data_test_dict_hor_2 = get_df_with_predictions_horizon(category_horizon_dict, test_dict_hor_2,2)
all_data_test_dict_hor_3 = get_df_with_predictions_horizon(category_horizon_dict, test_dict_hor_3,3)
all_data_test_dict_hor_4 = get_df_with_predictions_horizon(category_horizon_dict, test_dict_hor_4,4)
all_data_test_dict_hor_8 = get_df_with_predictions_horizon(category_horizon_dict, test_dict_hor_8,8)

In [20]:
all_data_test_dict_hor_2['All items']

,Category,Date,Year,Prediction Horizon 2,Actual Horizon 2
0,All items,2021-01-15,2021.0,0.235949,0.618238
1,All items,2021-02-15,2021.0,0.344591,0.767086
2,All items,2021-03-15,2021.0,0.488324,0.642159
3,All items,2021-04-15,2021.0,0.547848,0.900787
4,All items,2021-05-15,2021.0,0.526242,0.472715
5,All items,2021-06-15,2021.0,0.649123,0.273989
6,All items,2021-07-15,2021.0,0.479769,0.411588
7,All items,2021-08-15,2021.0,0.315077,0.938899
8,All items,2021-09-15,2021.0,0.389864,0.776096
9,All items,2021-10-15,2021.0,0.621463,0.469350


In [21]:
def plot_results_horizon(all_data_dict, categories, horizon):
    #category_samples = random.sample(categories, 5)+['All items']
    category_samples = categories
    for category in category_samples:
        category_df = all_data_dict[category]
        y_pred = category_df[f'Prediction Horizon {horizon}'].values
        y_actual = category_df[f'Actual Horizon {horizon}'].values

        non_null_indices = ~np.isnan(y_pred) & ~np.isnan(y_actual)

        predicted_values_non_null = y_pred[non_null_indices]
        observed_values_non_null = y_actual[non_null_indices]

        mse = mean_squared_error(predicted_values_non_null, observed_values_non_null)
        print(f'Category is: {category}')
        print(f'RMSE is: {np.sqrt(mse)}')

        fig = px.line(category_df, x="Date", y=[f'Actual Horizon {horizon}', f'Prediction Horizon {horizon}'], title=f'{category} - Actual VS Prediction: horizon {horizon}')
        fig.show()



In [22]:
def total_corr_horizon(all_data_test_dict, horizon):
    corr_dict = {}
    for key in list(all_data_test_dict.keys()):
        df = all_data_test_dict[key]
        y_pred = df[f'Prediction Horizon {horizon}'].values
        y_actual = df[f'Actual Horizon {horizon}'].values

        non_null_indices = ~np.isnan(y_pred) & ~np.isnan(y_actual)

        predicted_values_non_null = y_pred[non_null_indices]
        observed_values_non_null = y_actual[non_null_indices]

        corr = stats.pearsonr(predicted_values_non_null,observed_values_non_null)[0]
        corr_dict[key] =  corr
    
    total_corr = sum(corr_dict.values())
    
    num_high_corr = 0
    for category in corr_dict:
        if corr_dict[category] >= 0.5:
            num_high_corr +=1
    
    print(f'Number of categories with High Correlation: {num_high_corr}')
    print(f'Total Corr Horizon {horizon}, {total_corr}')
    return total_corr

In [29]:
Year

2020

In [23]:
total_corr_horizon(all_data_test_dict_hor_2, 2)
total_corr_horizon(all_data_test_dict_hor_3, 3)
total_corr_horizon(all_data_test_dict_hor_4, 4)
total_corr_horizon(all_data_test_dict_hor_8, 8)

Number of categories with High Correlation: 11
Total Corr Horizon 2, 11.267516831451786
Number of categories with High Correlation: 3
Total Corr Horizon 3, 4.336698427801739
Number of categories with High Correlation: 3
Total Corr Horizon 4, 2.2012058982074523
Number of categories with High Correlation: 13
Total Corr Horizon 8, 6.054677357918588


6.054677357918588

In [24]:
cats = list(all_data_test_dict_hor_2.keys())
categories = random.sample(cats, 5)+['All items']

In [25]:
#categories = list(all_data_test_dict_hor_2.keys())
plot_results_horizon(all_data_test_dict_hor_2, categories, 2)

Category is: Full service meals and snacks
RMSE is: 0.3014128856333011


Category is: Sugar and sweets
RMSE is: 0.9605413957920627


Category is: Rent of shelter
RMSE is: 0.15148865082591575


Category is: Crackers, bread, and cracker products
RMSE is: 2.2687887735971306


Category is: Motor vehicle maintenance and servicing
RMSE is: 0.6132843719158987


Category is: All items
RMSE is: 0.39667820502308504


In [26]:
#categories = list(all_data_test_dict_hor_3.keys())
plot_results_horizon(all_data_test_dict_hor_3, categories, 3)

Category is: Full service meals and snacks
RMSE is: 0.34406559528823516


Category is: Sugar and sweets
RMSE is: 0.9619230053063077


Category is: Rent of shelter
RMSE is: 0.17312349190391027


Category is: Crackers, bread, and cracker products
RMSE is: 2.0724829542876373


Category is: Motor vehicle maintenance and servicing
RMSE is: 0.6543818625014399


Category is: All items
RMSE is: 0.35486260489670396


In [27]:
#categories = list(all_data_test_dict_hor_4.keys())
plot_results_horizon(all_data_test_dict_hor_4, categories, 4)

Category is: Full service meals and snacks
RMSE is: 0.3432016420090018


Category is: Sugar and sweets
RMSE is: 0.9695535651984478


Category is: Rent of shelter
RMSE is: 0.17256391346832337


Category is: Crackers, bread, and cracker products
RMSE is: 2.1018216852742664


Category is: Motor vehicle maintenance and servicing
RMSE is: 0.695281397513591


Category is: All items
RMSE is: 0.38748887429813533


In [28]:
#categories = list(all_data_test_dict_hor_8.keys())
plot_results_horizon(all_data_test_dict_hor_8, categories, 8)

Category is: Full service meals and snacks
RMSE is: 0.37889397717521794


Category is: Sugar and sweets
RMSE is: 0.9843890406963314


Category is: Rent of shelter
RMSE is: 0.19339919547090698


Category is: Crackers, bread, and cracker products
RMSE is: 2.029724404840331


Category is: Motor vehicle maintenance and servicing
RMSE is: 0.6686234853832796


Category is: All items
RMSE is: 0.4172902098383722
